# Proof that PAV satisfies the core for $k = 7$

This notebook contains code for verifying the key inequality (3) appearing in the proof of Theorem 4.1. This inequality is stated for every possible deviation $T$ and ballot $A$:
$$
    \delta_A > (\tfrac{k}{|T|} - 1) \cdot |T \setminus W|.
$$
If this inequality is established for all $A$, then the proof implies that $T$ is not successful.

In [1]:
import itertools
from fractions import Fraction
from utility_functions import generate_wlog_partition, is_wlog

In this notebook, we refer to the quantity $\delta_A$ as the `LHS` (left-hand side). The following function computes it, in the simplified form given in the paper which depends on the following quantities:
$$
    a = (W \setminus T) \cap A, \quad
	b = (W \cap T) \cap A, \quad
	c = (T \setminus W) \cap A.
$$
as well as the cardinalities of $W \setminus T$ and $T \setminus W$.

In [2]:
def LHS(a, b, c, W_minus_T, T_minus_W):
    # delta_A in the paper
    if a + b:
        return Fraction((W_minus_T - a) * c, a + b + 1) - Fraction(a * (T_minus_W - c), a + b)
    else:
        # a + b = 0, so a = 0, so second term is 0 / 0 = 0
        return Fraction((W_minus_T - a) * c, a + b + 1)

Finally, the function `check_inequality` verifies the inequality for all possible deviations $T$ and ballots $A$, for given committee size $k$. To speed up the computation, it only considers canonical deviations $T$.

In [3]:
def check_inequality(k):
    violations = set()
    W = range(k) # committee, without loss of generality
    for T_size in range(1, k):
        wlog_partition = generate_wlog_partition(range(k + T_size), [range(k)])
        for T in itertools.combinations(range(k + T_size), T_size):
            if not is_wlog(T, wlog_partition):
                continue
            W_minus_T = sum(1 for w in W if w not in T)
            W_cap_T = sum(1 for w in W if w in T)
            T_minus_W = sum(1 for t in T if t not in W)
            if not T_minus_W:
                continue # cannot be a deviation
            for a, b, c in itertools.product(*[
                                    range(W_minus_T + 1), 
                                    range(W_cap_T + 1), 
                                    range(T_minus_W + 1)]):
                # a = number approved in W \ T
                # b = number approved in W cap T
                # c = number approved in T \ W
                if b + c > a + b: # prefers T to W
                    if LHS(a, b, c, W_minus_T, T_minus_W) <= (Fraction(k, len(T)) - 1) * T_minus_W:
                        violations.add(T)
    return violations

We run this function for each $k = 1, \dots, 7$, verifying that the inequality always holds. We also run it for $k = 8$, discovering that the inequality holds except for $T = \{0,1,8,9\}$, a fact that will be used in Lemma 4.4.

In [4]:
for k in range(1, 9):
    violations = check_inequality(k)
    if violations:
        print(f"Found violations for k = {k}: T ∈ {violations}")
    else:
        print(f"No violations found for k = {k}")

No violations found for k = 1
No violations found for k = 2
No violations found for k = 3
No violations found for k = 4
No violations found for k = 5
No violations found for k = 6
No violations found for k = 7
Found violations for k = 8: T ∈ {(0, 1, 8, 9)}
